# Извлечение ключевых слов

## 1.

В качестве корпуса мною были взяты тексты рецептов на русском языке с сайта gastronom.ru. 

На странице каждого рецепта уже указаны ключевые слова в виде перечисления через запятую. Предполагаю, что авторы рецептов делают это самостоятельно, потому что количество и смысл выделенных ключевых слов от рецепта к рецепту может сильно разниться. 

Пример : https://www.gastronom.ru/recipe/4632/sup-harcho-iz-govjadiny-s-risom

Я выбирала достаточно длинные рецепты, чтобы текст в них был не короче, чем 300 символов. Сам текст и ключевые слова доставала вручную (можно посмотреть код 'мини-корпус-рецептов.ipynb'). В выборку вошли 10 рецептов.

In [1]:
import pandas as pd

corpus = pd.read_csv('kw-recipes.csv')
corpus.head()

,text,keywords
0,Ризотто с морепродуктами — один из многочислен...,"морепродукты, рис, мидии, креветки, итальянска..."
1,\nСуп харчо из говядины с рисом — знаменитое б...,"говядина, рис, гранатовый сок, грузинская кухн..."
2,"\nРаньше такое блюдо, как фаршированный перец ...","говядина, баранина, мясной фарш, узбекская кух..."
3,Хорошо приготовленный плов с курицей в казане ...,"айва, плов, плов с курицей"
4,Как приятно в холодный пасмурный осенний день ...,"паста, лазанья, итальянская кухня, говядина"


Проверим, что примерный объем корпуса подходит под задание:

In [2]:
len(' '.join(corpus['text']).split())

4272

## 2.

Добавлю свою собственную разметку по ключевым словам:

In [3]:
corpus['keywords_hand'] = [
    'ризотто с морепродуктами, рис, морепродукты, итальянская кухня, оливковое масло, сухое вино',
    'суп харчо, говядина с рисом, грузинская кухня, бульон, гранатовый сок, зелень, приправы',
    'фаршированный перец, мясной фарш, жареные овощи, приправы',
    'плов с курицей, узбекская кухня, казан, чеснок, рис, зирвак, мясо',
    'итальянская кухня, лазанья, говядина, тесто, соус болоньезе, соус бешамель',
    'морковный пирог, морковь, изюм, грецкие орехи, тесто, миксер, выпечка',
    'кулич, тесто для кулича, глазурь, опара, миксер, выпечка',
    'заливное из курицы, праздничный стол, бульон, курица, вареное яйцо',
    'суп фо, вьетнамская кухня, рисовая лапша, овощи, бульон, утиная грудка',
    'торт птичье молоко, тесто, бисквит, глазурь, выпечка, миксер,'  
]

Преобразуем колонки с ключевыми словами из строк в списки для более удобной работы:

In [4]:
corpus['keywords'] = corpus['keywords'].apply(lambda x: x.split(', '))
corpus['keywords_hand'] = corpus['keywords_hand'].apply(lambda x: x.split(', '))

Посмотрим на пересечение моей и авторской разметки:

In [5]:
import numpy as np 

intersects = corpus.apply(lambda x: np.intersect1d(x['keywords'],x['keywords_hand']), axis=1)
for x in intersects:
    print(x)

['итальянская кухня' 'морепродукты' 'рис']
['гранатовый сок' 'грузинская кухня']
['мясной фарш' 'фаршированный перец']
['плов с курицей']
['говядина' 'итальянская кухня' 'лазанья']
['выпечка' 'грецкие орехи' 'изюм' 'морковный пирог' 'морковь']
['выпечка' 'кулич']
['заливное из курицы' 'курица' 'праздничный стол']
['рисовая лапша' 'суп фо' 'утиная грудка']
['торт птичье молоко']


В пересечение попали достаточно очевидные элементы текстов рецепта, например, почти везде попало название самого блюда, основные ингридиенты. А также можно увидеть, что и я и автор выделяли ключевыми либо национальную особенность (грузинская кухня) блюда либо способ приготовления (выпечка)

Я также не включала в разметку все ингридиенты блюда (в отличие от оригинальной, где часто перечислены каждый овощ, каждая специя), не указывала какие-то праздники или события, с чем блюдо ассоциируется (например, ужин, новый год или пасха), поскольку этого прямого указания в тексте нет.

Есть три текста, где пересечением является только одно слово, и в целом размер пересечения не очень большой (2-3 слова в среднем), поэтому в качестве эталона логичнее взять объединение, чтобы было больше материала для дальнейшей работы.

In [6]:
corpus['etalon'] = corpus.apply(lambda x: np.union1d(x['keywords'],x['keywords_hand']), axis=1)

## 3.

Напишем сразу функцию предобработки, которая будет делить тексты на токены, приводить слова к нижнему регистру, убирать знаки пунктуации и совершать лемматизацию

In [7]:
import pymorphy2
import nltk
from string import punctuation
from nltk.corpus import stopwords

In [11]:
nltk.download('stopwords')
stopwords = stopwords.words('russian')
punkt = punctuation + '«»—…“”*№–'
morph = pymorphy2.MorphAnalyzer()

def preprocessing(text):
    tokens = nltk.word_tokenize(text)
    tokens = [_.lower() for _ in tokens if _.lower() not in punkt]
    lemmas = [morph.parse(word)[0].normal_form for word in tokens]
    return ' '.join(lemmas)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**RAKE**

In [9]:
!pip install python-rake

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [12]:
import RAKE

rake = RAKE.Rake(stopwords)
preprocessing(corpus['text'][0])
#посмотрим на пример
rake.run(preprocessing(corpus['text'][0]), maxWords=3, minFrequency=2)

[('процесс приготовление', 4.0),
 ('пора пока', 4.0),
 ('кой случай', 3.666666666666667),
 ('ризотто', 1.75),
 ('сразу', 1.6666666666666667),
 ('морепродукт', 1.25),
 ('слегка', 1.0)]

In [128]:
corpus['keywords_rake'] = corpus['text'].apply(lambda x: rake.run(preprocessing(x), maxWords=3, minFrequency=2))
#вытащим из списка кортежей только сами слова
corpus['keywords_rake'] = corpus['keywords_rake'].apply(lambda x: [i[0] for i in x])

**TextRank**

In [ ]:
!pip install summa

In [14]:
from summa import keywords

# посмотрим на примере
keywords.keywords(preprocessing(corpus['text'][0]), language='russian', additional_stopwords=stopwords, scores=True)

[('ризотто', 0.3077494106629952),
 ('лука', 0.1563321954067641),
 ('чеснок раздавить', 0.13750577078587095),
 ('это', 0.13176819900799153),
 ('морепродукт', 0.12891262347222157),
 ('время', 0.12095052285931167),
 ('всыпать сухой рис', 0.11809484033176383),
 ('половник', 0.11659711360352569),
 ('должный', 0.11527155389251971),
 ('нужный', 0.11414439043149406),
 ('нужно', 0.11414439043149406),
 ('мина добавить', 0.11020769568870238),
 ('очень', 0.10970433143465408),
 ('точно следовать', 0.10616003250389303),
 ('запах', 0.10009118803543124),
 ('нарезать', 0.0991600271834301),
 ('заранее', 0.0906186966166167),
 ('общий', 0.0902252318548854),
 ('risotto', 0.0898258469844134),
 ('белый перец', 0.08969561880865534),
 ('итальянский блюдо кстати', 0.08918977464491791),
 ('приготовление', 0.0882251925961469),
 ('который', 0.08820660345658153),
 ('белые', 0.08784919408345128),
 ('пристальный', 0.08641003609035122),
 ('костный', 0.08641003609035065),
 ('колбасный', 0.0864100360903504),
 ('каждый',

In [15]:
corpus['keywords_textrank'] = corpus['text'].apply(lambda x: keywords.keywords(preprocessing(x), 
                                                                     language='russian',
                                                                     additional_stopwords=stopwords).split())

Большую частотность, судя по всему, имеет слово *это*, но оно не имеет конкретного значения и входит в разряд служебных в рамках этой задачи

In [16]:
stopwords.append('это')

**TF-IDF**

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

Для каждого текста в корпусе мы:

создаем матрицу tf-idf (в данном случае, вектор) для униграмм, биграмм и трехграмм текста, сортируем вектор по убыванию значений и получаем индексы, по первым 10 индексам получаем сами слова из словаря

In [18]:
kw_tfidf = []
vec = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,3))
vec.fit(corpus['text'])
words = vec.get_feature_names()
for t in corpus['text']:
    X = vec.transform([preprocessing(t)])
    words = vec.get_feature_names()
    inds = np.flip(np.argsort(X.toarray())[::-1])[0][:10]
    kw = [words[i] for i in inds]
    kw_tfidf.append(kw)

In [19]:
kw_tfidf[0]

['ризотто',
 'рис',
 'добавить',
 'половник',
 'раздавить',
 'лука',
 'время',
 'чеснок',
 'бульон',
 'кстати']

In [20]:
corpus['keywords_tfidf'] = kw_tfidf

## 4.

Чтобы работать с синтаксисом и морфологией, использую библиотеку udpipe (я уже много раз ей пользовалась и мне кажется невозможным использовать ее так, как показали в семинаре через командную строку)

In [23]:
import wget
from ufal.udpipe import Model, Pipeline

wget.download('https://rusvectores.org/static/models/udpipe_syntagrus.model')
m = Model.load('udpipe_syntagrus.model')
process_pipeline = Pipeline(m, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

100% [........................................................................] 40616122 / 40616122

В качестве ключевых слов буду оставлять либо существительные либо именные группы существительное+прилагательное

Для этого воспользуюсь представлением в виде деревьев от NLTK 

In [100]:
from nltk.parse import DependencyGraph


def filtering(g):
    """
    Функция, проходящаяся по всем узлам дерева и возвращает либо существительное либо
    существительное и от него зависящее прилагательное (если такое есть)"""
    result = []
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'NOUN':
            if 'amod' in g.nodes[n]['deps']:
                adj_ind = g.nodes[n]['deps']['amod'][0]
                result.append(' '.join((g.nodes[adj_ind]['lemma'].lower(), g.nodes[n]['lemma'].lower())))
            else:
                result.append(g.nodes[n]['lemma'].lower())
    return result

In [101]:
from nltk.parse import DependencyGraph

# будем записывать отфильтрованные сущности для каждого текста
filter_np = []
# проходимся по каждому тексту
for text in corpus['text']:
    entities = []
    # делим текст на предложения, поскольку дерево строится только для предложений, а не всего текста
    for sent in nltk.sent_tokenize(text):
        # обрабатываем предложение с помощью udpipe
        processed = process_pipeline.process(sent)
        # парсим полученную информацию и создаем на ее основе дерево
        content = [l for l in processed.split('\n') if not l.startswith('#')]
        g = DependencyGraph(content, top_relation_label='root')
        #сохраняем найденные элементы
        entities += filtering(g)
    filter_np.append(entities)

Посмотрим, что получилось на примере текста про ризотто:

In [102]:
filter_np[0][:20]

['ризотто',
 'морепродукт',
 'многочисленный вариант',
 'знаменитый блюдо',
 'сравнение',
 'сало',
 'костный мозг',
 'сыр',
 'мясо',
 'колбасный изделие',
 'фасоль',
 'главное',
 'правильный морепродукт',
 'счет',
 'вкус',
 'блюдо',
 'аромат',
 'достижение',
 'идеальный результат',
 'указание']

Все хорошо!

Теперь для каждого текста найдем пересечение отфильтрованных единиц и найденных ключевых слов

In [107]:
import numpy as np

corpus['filter'] = filter_np

corpus['f_keywords_rake'] = corpus.apply(lambda x: np.intersect1d(x['keywords_rake'], x['filter']), axis=1)
corpus['f_keywords_textrank'] = corpus.apply(lambda x: np.intersect1d(x['keywords_textrank'], x['filter']), axis=1)
corpus['f_keywords_tfidf'] = corpus.apply(lambda x: np.intersect1d(x['keywords_tfidf'], x['filter']), axis=1)

Посмотрим, что получилось

In [108]:
corpus[['f_keywords_rake', 'f_keywords_textrank', 'f_keywords_tfidf']]

,f_keywords_rake,f_keywords_textrank,f_keywords_tfidf
0,"[морепродукт, ризотто]","[блюдо, бульон, вино, время, запах, лука, миди...","[бульон, время, лука, половник, ризотто, рис, ..."
1,"[говядина, харчо]","[бульон, вкус, говядина, кастрюля, корень, мин...","[бульон, огонь, рис, суп, харчо]"
2,[мясо],"[кубик, луковица, минута, морковь, начинка, ов...","[начинка, перец, рис, сладкий перец, фарш, фар..."
3,"[курица, плов]","[айва, вода, зирвак, казан, курица, кусок, лук...","[вода, зирвак, казан, курица, мясо, огонь, пло..."
4,[],"[ингредиент, конец, кусочек, минута, размер, с...","[кусочек, слой, соус, тесто]"
5,[форма],"[духовка, изюм, корнеплод, лопаточка, минута, ...","[морковный пирог, морковь, пирог]"
6,"[кулич, форма]","[дрожжи, духовка, кулич, литр, минута, мука, п...","[дрожжи, кулич, полотенце, тесто]"
7,"[балкон, бульон, крышка, курица, небольшой ого...","[бульон, вода, время, желатин, кастрюля, колич...","[бульон, вода, курица, литр, ложка, мясо, яйцо]"
8,[чеснок],"[бульон, бутон, говядина, грудка, жир, кастрюл...","[бульон, кипение, огонь, чеснок]"
9,"[форма, холодильник]","[бисквит, год, конфета, крахмал, крем, масса, ...","[крем, масса, молоко, основа, торт]"


Из-за того, что RAKE выделил не очень много ключевых слов, и какие-то из них отфильтровались, то их осталось совсем мало

В результатах, полученными другими алгоритмами, все хорошо, осталось много элементов, ушли ненужные глаголы, которые хоть и часто встречаются в текстах рецептов, ключевыми не являются

## 5.

Напишем функции, которые будут считать необходимые метрики


precision - отношение верно найденных алгоритмом ключевых слов ко всем найденным алгоритмом ключевым словам


recall - отношение верно найденных алгоритмом ключевых слов ко всем ключевым словам, которые должны были быть найдены

In [142]:
def precision(kw_etalon, kw_pred):
    if len(kw_pred)==0:
        return 0
    return np.intersect1d(kw_etalon, kw_pred).shape[0]/len(kw_pred)


def recall(kw_etalon, kw_pred):
    if len(kw_etalon)==0:
        return 0
    return np.intersect1d(kw_etalon, kw_pred).shape[0]/len(kw_etalon)


def f1(kw_etalon, kw_pred):
    p = precision(kw_etalon, kw_pred)
    r = recall(kw_etalon, kw_pred)
    if p==0 or r==0:
        return 0
    return 2*p*r/(p+r)

Еще необходимо учесть, что выделенные мной и авторами рецептов ключевые слова не стоят в начальной форме и содержат стоп-слова (типа "плов с курицей"). Выделенные алгоритмами списки слов этого не содержат

In [119]:
def kw_cleaning(words):
    clean_words = []
    for ngram in words:
        new_ngram = []
        for word in ngram.split():
            if word.lower() not in stopwords:
                new_ngram.append(morph.parse(word)[0].normal_form)
        clean_words.append(' '.join(new_ngram))
    return clean_words

In [120]:
kw_cleaning(['плов с курицей'])

['плов курица']

Посчитаем метрики для каждого текста нашего корпуса и усредним:

In [145]:
print('Precision for RAKE: ', corpus.apply(lambda x: precision(kw_cleaning(x['etalon']), x['keywords_rake']), axis=1).mean())
print('Recall for RAKE: ', corpus.apply(lambda x: recall(kw_cleaning(x['etalon']), x['keywords_rake']), axis=1).mean())
print('f1-score for RAKE: ', corpus.apply(lambda x: f1(kw_cleaning(x['etalon']), x['keywords_rake']), axis=1).mean())
print('\n\n\n')
print('Precision for RAKE with filters: ', corpus.apply(lambda x: precision(kw_cleaning(x['etalon']), x['f_keywords_rake']), axis=1).mean())
print('Recall for RAKE with filters: ', corpus.apply(lambda x: recall(kw_cleaning(x['etalon']), x['f_keywords_rake']), axis=1).mean())
print('f1-score for RAKE with filters: ', corpus.apply(lambda x: f1(kw_cleaning(x['etalon']), x['f_keywords_rake']), axis=1).mean())

Precision for RAKE:  0.205
Recall for RAKE:  0.08688034188034188
f1-score for RAKE:  0.11414427076191783




Precision for RAKE with filters:  0.3333333333333333
Recall for RAKE with filters:  0.07085470085470086
f1-score for RAKE with filters:  0.11373737373737373


In [146]:
print('Precision for TextRank: ', corpus.apply(lambda x: precision(kw_cleaning(x['etalon']), x['keywords_textrank']), axis=1).mean())
print('Recall for TextRank: ', corpus.apply(lambda x: recall(kw_cleaning(x['etalon']), x['keywords_textrank']), axis=1).mean())
print('f1-score for TextRank: ', corpus.apply(lambda x: f1(kw_cleaning(x['etalon']), x['keywords_textrank']), axis=1).mean())
print('\n\n\n')
print('Precision for TextRank with filters: ', corpus.apply(lambda x: precision(kw_cleaning(x['etalon']), x['f_keywords_textrank']), axis=1).mean())
print('Recall for TextRank with filters: ', corpus.apply(lambda x: recall(kw_cleaning(x['etalon']), x['f_keywords_textrank']), axis=1).mean())
print('f1-score for TextRank with filters: ', corpus.apply(lambda x: f1(kw_cleaning(x['etalon']), x['f_keywords_textrank']), axis=1).mean())

Precision for TextRank:  0.07503353290422646
Recall for TextRank:  0.2967959817959818
f1-score for TextRank:  0.1179158796595053




Precision for TextRank with filters:  0.20357975357975358
Recall for TextRank with filters:  0.2658436008436008
f1-score for TextRank with filters:  0.22324645821576303


In [147]:
print('Precision for TF-IDF: ', corpus.apply(lambda x: precision(kw_cleaning(x['etalon']), x['keywords_tfidf']), axis=1).mean())
print('Recall for TF-IDF: ', corpus.apply(lambda x: recall(kw_cleaning(x['etalon']), x['keywords_tfidf']), axis=1).mean())
print('f1-score for TF-IDF: ', corpus.apply(lambda x: f1(kw_cleaning(x['etalon']), x['keywords_tfidf']), axis=1).mean())
print('\n\n\n')
print('Precision for TF-IDF with filters: ', corpus.apply(lambda x: precision(kw_cleaning(x['etalon']), x['f_keywords_tfidf']), axis=1).mean())
print('Recall for TF-IDF with filters: ', corpus.apply(lambda x: recall(kw_cleaning(x['etalon']), x['f_keywords_tfidf']), axis=1).mean())
print('f1-score for TF-IDF with filters: ', corpus.apply(lambda x: f1(kw_cleaning(x['etalon']), x['f_keywords_tfidf']), axis=1).mean())

Precision for TF-IDF:  0.22999999999999998
Recall for TF-IDF:  0.22425269175269177
f1-score for TF-IDF:  0.22514770224017772




Precision for TF-IDF with filters:  0.40436507936507937
Recall for TF-IDF with filters:  0.20822705072705072
f1-score for TF-IDF with filters:  0.26381428169353865


## 6.

Одну из проблем автоматических подходов к выделению ключевых слов мы решили - отфильтровали нужное с помощью морфосинтаксических признаков. Поскольку алгоритмы учитывают частоту встречаемости, то не обращает внимания на части речи (только в textrank от gensim можно сразу передать pos-фильтры). Это улучшило результаты достаточно значительно, потому что в рецептах часто встречаются различные глаголы действия (типа *перемешайте*, *посолите* и т.д.)


Также и мной и авторами рецептов в списки ключевых слов были включены, по сути, заголовки - полные названия блюд (торт птичье молоко). Алгоритмы этого не делают и делят это значимое словосочетание на различные единицы. Наверное, чтобы этого избежать, надо каким-то образом повысить вес именно этого словосочетания вручную


Основная проблема, которая свойственна, наверное, всем алгоритмам, - трудности с обобщением. То есть, например, прочитав какой-либо рецепт любой человек способен сказать, это "второе блюдо", "угощение на новый год", или "холодная закуска", даже если этих слов в тексте нет? Поэтому подобного рода вещи в ключевые слова не включаются алгоритмами. Решить эту проблему достаточно трудно, нужна какая-то внешняя база знаний, которая будет приниматься во внимание


Еще проблема которая может возникнуть, но именно в рамках моего датасета ее не было: алгоритм не включает в ключевые слова важные именованные сущности, например, имена или географические названия, потому что они, допустим, встретились только в вводном предложении. В таком случае нужно подключать инструменты для задачи NER и вручную приписывать им больший вес или автоматически заносить в ключевые слова

